In [ ]:
import pandas as pd
import string
import base64
import re

In [ ]:
def decode_base64(word):
    try:
        return base64.b64decode(word).decode("utf-8")
    except:
        pass

In [ ]:
def split_session(full_session):
    words = []
    for word in re.split(r"\n|;|,|/|-|\||=|$|>|<|$|:|{|}|\(|\)| ", full_session):
        if word.startswith('"') or word.endswith('"'):
            # remove the quotation mark at the start and at the end of the word
            word = word[1:-1]
        elif len(word) == 1 and word in string.punctuation:
            # remove that punctuation
            word = None
        words.append(word)
    return list(filter(None, words))

In [ ]:
def clean_session(full_session):
    new_full_session = split_session(full_session)
    for i,word in enumerate(new_full_session):
        word_decoded = decode_base64(word)
        if word_decoded:
            list_word_decoded = split_session(word_decoded)
            new_full_session.pop(i)
            for j, d in enumerate(list_word_decoded):
                new_full_session.insert(i + j, d)
            break
        else:
            new_full_session.append(word)
    return new_full_session

In [ ]:
df_original = pd.read_parquet('ssh_attacks.parquet')
df=df_original.copy()
df['first_timestamp'] = pd.to_datetime(df['first_timestamp'])
df